<a href="https://colab.research.google.com/github/Magaton1010/image-analysis/blob/main/paper_Thresholding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

let's integrate the code details into the provided text:

**Automated Plant Segmentation in Agricultural Images Using Computer Vision Techniques**

**For abstract part :** introduces an automated approach to segment plant regions in agricultural images through computer vision methods. The objective is to precisely identify and measure plant areas for agricultural analysis. The proposed method integrates image processing techniques to isolate green plant pixels, detect plant structures, and calculate the pixel count, which can be converted to real-world units based on Ground Control Points (GCP) and the calculation for GSD that has been done to get the scale for the real world measurement scale.

** For introduction:** The growing demand for precision agriculture requires advanced tools for accurate and efficient plant analysis. Computer vision techniques provide promising solutions for automating the segmentation of plant regions in agricultural images. The goal is to streamline the process of quantifying plant areas, enabling farmers to make informed decisions, and researchers to conduct further analysis.

** For methodology:**

**2.1 Data Collection:** A dataset of agricultural images was collected for evaluation. Images were acquired using a standardized imaging setup with a drone named {name} from 150 feet to ensure consistency. Ground Control Points (GCP) were strategically placed to enable validation. The Ground Sampling Distance (GSD), a crucial parameter for accurate measurements, was computed based on the focal length of the camera, the altitude of the drone, and the image resolution. This calculation ensures that the pixel count in the segmented plant area can be accurately converted to real-world units, providing a scale for precise agricultural analysis.

**2.2 Image Processing Pipeline:** The proposed pipeline involves several key steps:

**Step 2.1: Thresholding:** Utilizing the HSV color space, green plant pixels are isolated using thresholding based on their distribution. The distribution analysis considers the range of pixel values for each image, determining upper and lower boundaries specific to the agricultural scene.

**Step 2.2: Edge Detection:** Plant structures are highlighted through edge detection. The Canny edge detection algorithm is employed for its effectiveness in detecting edges while minimizing noise.

**Step 2.3: Mask Creation:** A binary mask is generated based on detected edges. This binary mask effectively isolates the regions of interest, emphasizing the plant structures.

**Step 2.4: Connected Components Labeling:** Plant regions are labeled using connected components analysis. This step aids in distinguishing individual plant areas and assigning unique labels to each connected component.

**Step 2.5: Pixel Counting:** The total pixel count in the segmented plant area is calculated. This involves summing the binary mask pixels corresponding to the segmented plant region.

**Step 2.6: Contour Detection:** Contours are identified to delineate the boundaries of segmented regions. These contours provide a visual representation of the segmented plant areas.

**Step 2.7: Contour Filtering:** Contours are filtered based on a minimum area threshold to refine the segmentation. This filtering step helps eliminate small contours that may not represent significant plant regions.

**2.3 Visualization:** The pipeline includes visualization steps to enhance the interpretability of the segmentation results. The original image is overlaid with contours outlining the segmented plant regions.

**For experimental Results:** The proposed method was applied to a set of agricultural images, specifically plastic mulched tomato crops for each row separately. The results were visually and quantitatively assessed. Segmentation accuracy was measured by comparing the algorithm's output with manual annotations.

**For discussion:** The automated plant segmentation method demonstrated promising results in accurately identifying and quantifying plant areas. The integration of connected components analysis and contour filtering contributed to robust segmentation, even in complex agricultural scenes.

**For conclusion:** This study presents a comprehensive computer vision-based approach for automated plant segmentation in agricultural images. The proposed method effectively identifies and quantifies plant areas, offering potential applications in precision agriculture and crop monitoring.

** For future Work:** Future work could explore the adaptation of the proposed method for real-time applications and evaluate its performance on diverse agricultural datasets. Additionally, considering the dynamic nature of plant growth, incorporating temporal analysis could enhance the accuracy of plant area quantification over time.

In [ ]:
# Import necessary libraries
import cv2  # OpenCV for image processing
import numpy as np  # NumPy for numerical operations
import matplotlib.pyplot as plt  # Matplotlib for visualization
import os  # Operating System module for file and folder operations
import pandas as pd  # Pandas for data manipulation and analysis

# Folder paths
input_folder = r''  # Path to input images
output_folder = r''  # Path to save segmented images
excel_file_path = r'file_name.xlsx'  # Path to save results in Excel

# Create output folders if they don't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lists to store results for Excel
image_names = []  # List to store image names
pixel_counts = []  # List to store pixel counts
segmented_areas_acres = []  # List to store segmented areas in acres

# Process each image file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        # Load the image
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        # Step 1: Thresholding to isolate green plant pixels
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        lower_green = np.array([35, 40, 40])
        upper_green = np.array([80, 255, 255])
        mask = cv2.inRange(hsv, lower_green, upper_green)

        # Step 2: Edge Detection to highlight plant structures
        gray_thresh = cv2.cvtColor(cv2.bitwise_and(image, image, mask=mask), cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray_thresh, 50, 150)

        # Step 3: Create Mask from edges
        mask = np.zeros_like(gray_thresh)
        mask[edges != 0] = 255

        # Step 4: Labeling connected components in the mask
        _, labeled_mask = cv2.connectedComponents(mask)

        # Step 6: Calculate the total pixel count in the segmented plant area
        segmented_plant_area_pixels = np.sum(mask == 255)

        # Step 8: Contour Detection to find boundaries of segmented regions
        contours, _ = cv2.findContours(labeled_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Step 9: Filter and Process Contours based on area
        min_contour_area = 1
        enclosed_pixels = 0
        for contour in contours:
            if cv2.contourArea(contour) > min_contour_area:
                enclosed_pixels += cv2.boundingRect(contour)[2] * cv2.boundingRect(contour)[3]

        # Step 10: Visualization - Overlay contours on the original image
        image_with_contours = image.copy()
        cv2.drawContours(image_with_contours, contours, -1, (255, 0, 0), 2)

        # Visualization - Display the image with contours
        plt.imshow(cv2.cvtColor(image_with_contours, cv2.COLOR_BGR2RGB))
        plt.title(f'Segmentation Visualization - {filename}')
        plt.show()

        # Save the segmented image with contours
        cv2.imwrite(os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_contours.tif"), image_with_contours)

        # Add results to lists for Excel
        image_names.append(filename)
        pixel_counts.append(segmented_plant_area_pixels)
        segmented_areas_acres.append(enclosed_pixels * ((2.0 ** 2) / (40 * 40 * 43560)))

# Create a DataFrame to store the results
df = pd.DataFrame({
    'Image Name': image_names,
    'Pixel Count': pixel_counts,
    'Segmented Area (acres)': segmented_areas_acres
})

# Save the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)